<div align="center">
    <h1>DS-210: Programming for Data Science</h1>
    <h1>Lecture 33</h1>
</div>

# 1. Closures (anonymous functions)
# 2. Iterators
# 3. Iterator + closure magic
# 4. How about Python?

# Course Evaluation

## https://bu.campuslabs.com/courseeval/



# <font color="red">1. Closures (anonymous functions)</font>
# 2. Iterators
# 3. Iterator + closure magic
# 4. How about Python?

## Closures (anonymous functions)

We have seen them before in Python (as lambda functions):

```Python
lambda a b: a * b
```

In Rust (with implicit or explicit type specification):
```
|a, b| a * b
|a: i32, b: i32| -> i32 {a * b}
```

In [2]:
{
  let f = |a, b| a * b;
  let x = 10;
  let y = 20;
  println!("{}",f(x,y));
};

200


## Sample application: lazy evaluation of a value
Compute a value only if needed

In [3]:
// What does it compute?
fn expensive_function(i:u32) -> u128 {
    if i <= 1 {
        i as u128
    } else {
        expensive_function(i-1) + expensive_function(i-2)
    }
}

In [4]:
expensive_function(44)

701408733

In [6]:
// This function always computes expensive_function(44), even if not needed.
// Method unwrap_or takes a default value as a parameter.
fn value_or_fib44(input:Option<u128>) -> u128 {
    input.unwrap_or(expensive_function(44))
}

In [12]:
use std::time::SystemTime;
let d = SystemTime::now();
// slow
value_or_fib44(None);
let elapsed = d.elapsed().unwrap().as_millis();
println!("{}", elapsed);

3069


In [14]:
let d = SystemTime::now();
// slow
value_or_fib44(Some(123));
let elapsed = d.elapsed().unwrap().as_millis();
println!("{}", elapsed);

3052


## Sample application: lazy evaluation of a value
Compute a value only if needed

In [17]:
// This function computes expensive_function(44) only if needed.
// Method unwrap_or_else's parameter is a function that computes
// the default value, not the default value itself. 
fn value_or_fib44_version_2(input:Option<u128>) -> u128 {
    input.unwrap_or_else(|| expensive_function(44))
}

In [18]:
// slow
let d = SystemTime::now();
value_or_fib44_version_2(None);
let elapsed = d.elapsed().unwrap().as_millis();
println!("{}", elapsed);

3049


In [19]:
// fast
let d = SystemTime::now();
value_or_fib44_version_2(Some(1));
let elapsed = d.elapsed().unwrap().as_millis();
println!("{}", elapsed);

0


* This programing pattern appears in many places.
* Another example: default value for an entry in HashMap

In [11]:
let mut map = std::collections::HashMap::<i32,i32>::new();
map.insert(1, 1);
*map.entry(1).or_insert_with(|| expensive_function(44) as i32) *= -1;
*map.entry(2).or_insert_with(|| expensive_function(44) as i32) *= -1;
println!("{}:{:?}    {}:{:?}",1,map.get(&1),2,map.get(&2));

1:Some(-1)    2:Some(-701408733)


# 1. Closures (anonymous functions)
# <font color="red">2. Iterators</font>
# 3. Iterator + closure magic
# 4. How about Python?

## Iterators

* provide values one by one
* method `next` provides next one
* `Some(value)` or `None` if no more available

Some ranges are iterators:
* `1..100`
* `0..`

First value has to be known (so `..` and `..123` are not)

In [9]:
let mut iter = 1..3; // must be mutable

In [12]:
println!("{:?}", iter.next());
println!("{:?}", iter.next());
println!("{:?}", iter.next());
println!("{:?}", iter.next());
println!("{:?}", iter.next());

Some(1)
Some(2)
None
None
None


## Iterator from scratch: implement trait `Iterator`

In [14]:
struct Fib {
    current: u128,
    next: u128,
}

impl Fib {
    fn new() -> Fib {
        Fib{current: 0, next: 1}
    }
}

impl Iterator for Fib {
    type Item = u128;
    
    fn next(&mut self) -> Option<Self::Item> {
        let now = self.current;
        self.current = self.next;
        self.next = now + self.current;
        Some(now)
    }
}

In [15]:
let mut fib = Fib::new();
for _ in 0..10 {
    print!("{:?} ",fib.next().unwrap());
}
println!();

0 1 1 2 3 5 8 13 21 34 


## Iterators come with many useful functions implemented

* `next() -> Get the next element of an iterator (None if there isn't one)`
* `collect() -> Put iterator elements in vector`
* `take(N) -> take first N elements of an iterator and turn them into an iterator`
* `cycle() -> Turn a finite iterator into an infinite one that repeats itself`
* `for_each(||, ) -> Apply a closure to each element in the iterator`
* `filter(||, ) -> Create new iterator from old one for elements where closure is true`
* `map(||, ) -> Create new iterator by applying closure to input iterator`
* `any(||, ) -> Return true if closure is true for any element of the iterator`
* `fold(a, |a, |, ) -> Initialize expression to a, execute closure on iterator and accumulate into a`
* `reduce(|x, y|, ) -> Similar to fold but the initial value is the first element in the iterator`
* `zip(iterator) -> Zip two iterators together to turn them into pairs`

`collect` can be used to put elements of an iterator into a vector:

In [16]:
let small_numbers : Vec<_> = (1..=10).collect();
small_numbers

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

`take` turns an iterator into an iterator that provides at most a specific number of elements

In [17]:
let small_numbers : Vec<_> = (1..).take(10).collect();
small_numbers

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

`cycle` creates an iterator that repeats itself forever:

In [18]:
let cycle : Vec<_> = (1..4).cycle().take(20).collect();
cycle

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2]

# 1. Closures (anonymous functions)
# 2. Iterators
# <font color="red">3. Iterator + closure magic</font>
# 4. How about Python?

## Iterator + closure magic
* Operate on entire sequence, sometimes lazily by creating a new iterator
* Allows for concise expression of many concepts

`for_each` applies a function to each element

In [19]:
(0..5).for_each(|x| println!("{}",x));

0
1
2
3
4


`filter` creates a new iterator that has elements for which the given function is true

In [3]:
let not_divisible_by_3 : Vec<_> = (0..10).filter(|x| x % 3 != 0).collect();
not_divisible_by_3

[1, 2, 4, 5, 7, 8]

## Iterator + closure magic
* Operate on entire sequence, sometimes lazily by creating a new iterator
* Allows for concise expression of many concepts


`map` creates a new iterator in which values are processed by a function

In [2]:
let fibonacci_squared : Vec<_> = Fib::new().take(10).map(|x| x*x).collect();
fibonacci_squared

Error: failed to resolve: use of undeclared type `Fib`

## Primes
`any` is true if the passed function is true on some element

Is a number prime?

In [22]:
fn is_prime(k:u32) -> bool {
    !(2..k).any(|x| k % x == 0)
}

In [25]:
is_prime(31)

true

Create infinite iterator over primes:

In [13]:
{
  let primes = (2..).filter(|k| !(2..*k).any(|x| k % x == 0));
  let v : Vec<_> = primes.take(200).collect();
  v
}

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997, 1009, 1013, 1019, 1021, 1031, 1033, 1039, 1049, 1051, 1061, 1063, 1069, 1087, 1091, 1093, 1097, 1103, 1109, 1117, 1123, 1129, 1151, 1153, 1163, 1171, 1181, 1187, 1193, 1201, 1213, 1217, 12

## Functional programming classics: `fold` and `reduce`

`iterator.fold(init,f)` equivalent to
```rust
let mut accumulator = init;
while let Some(x) = iterator.next() {
    accumulator = f(accumulator,x);
}
accumulator
```

**Example:** compute $\sum_{i=1}^{10} x^2$

In [27]:
let sum_of_squares: i32 = (1..=10).fold(0,|a,x| a + x * x);
sum_of_squares

385

In [28]:
// Another approach: using `sum` (which can be implemented using `fold`)
let sum_of_squares: i32 = (1..=10).map(|x| x * x).sum();
sum_of_squares

385

## Functional programming classics: `fold` and `reduce`

`iterator.reduce(f)` equivalent to
```rust
if let Some(x) = iterator.next() {
    let mut accumulator = x;
    while let Some(y) = iterator.next() { accumulator = f(accumulator,y}
    Some(accumulator)
} else {
    None
}
```

Differences from `fold`:
* no default value for an empty sequence 
* output must be the same type as elements of input sequence
* output for length–one sequence equals the only element in the sequence

## Functional programming classics: `fold` and `reduce`

`iterator.reduce(f)` equivalent to
```rust
if let Some(x) = iterator.next() {
    let mut accumulator = x;
    while let Some(y) = iterator.next() { accumulator = f(accumulator,y) }
    Some(accumulator)
} else {
    None
}
```

**Example:** computing the maximum number in $\{x^2 \bmod 7853: x \in [123] \}$

In [29]:
(1..=123).map(|x| (x*x) % 7853).reduce(|x,y| x.max(y)).unwrap()

7744

In [30]:
// in this case one can use the builtin `max` method (which can be implemented, using `fold`)
(1..=123).map(|x| (x*x) % 7853).max().unwrap()

7744

## Combining two iterators: `zip`

* Returns an iterator of pairs
* The length is the minimum of the lengths

In [31]:
let v: Vec<_> = (1..10).zip(11..20).collect();
v

[(1, 11), (2, 12), (3, 13), (4, 14), (5, 15), (6, 16), (7, 17), (8, 18), (9, 19)]

Inner product of two vectors:

In [32]:
let x: Vec<f64> = vec![1.1,  2.2, -1.3,  2.2];
let y: Vec<f64>  = vec![2.7, -1.2, -1.1, -3.4];
let inner_product: f64 = x.iter().zip(y.iter()).map(|(a,b)| a * b).sum();
inner_product

-5.72

# 1. Closures (anonymous functions)
# 2. Iterators
# 3. Iterator + closure magic
# <font color="red">4. How about Python?</font>

<div align="center">
    <h1>[Switch to the Python notebook]</h1>
</div>